# Generate Data

In [58]:
import numpy as np
import causaldata
import math
from doubleml.datasets import make_plr_CCDDHNR2018, make_plr_turrell2018
np.random.seed(1234)
n_rep = 1
n_obs = 10000
n_vars = 10
alpha = 0.5
data = list()
from sklearn.datasets import make_spd_matrix

def g(x):
    return np.exp(x)

def m(x):
    return np.sin(x)

theta = alpha = 0.5 
b = [1/k for k in range(1,n_vars+1)] # x weights 
sigma = make_spd_matrix(n_vars, random_state=42)

for i_rep in range(n_rep):
    import pandas as pd
    import numpy as np
    df = causaldata.thornton_hiv.load_pandas().data
    df = df.dropna()
    outcome = 'got'
    treatment = 'any'
    rest = df.drop([outcome, treatment, 'villnum',  'tinc'], axis=1).columns
    y = df[outcome]
    d = df[treatment]
    x = df[rest].astype('float')
    print(y.shape, x.shape, d.shape)
    x.head()
    data.append((x, y, d))

(2825,) (2825, 3) (2825,)


In [67]:
rest

Index(['distvct', 'age', 'hiv2004'], dtype='object')

In [60]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
OLS_Y = LinearRegression()
OLS_D = LogisticRegression()
RF_Y = RandomForestRegressor(n_estimators = 5)
RF_D = RandomForestClassifier(n_estimators = 5)

from prettytable import PrettyTable
from PIL import Image, ImageDraw, ImageFont
table = PrettyTable()
table.field_names = ['Model', 'OLS', 'RF']
a = ['Y on X (R2)'] + [OLS_Y.fit(x,y).score(x,y), RF_Y.fit(x,y).score(x,y)]
table.add_row(a)
a = ['D on X (Accuracy)'] + [OLS_D.fit(x,d).score(x,d), RF_D.fit(x,d).score(x,d)]
table.add_row(a)
table.float_format = '0.3'
print(table)

+-------------------+-------+-------+
|       Model       |  OLS  |   RF  |
+-------------------+-------+-------+
|    Y on X (R2)    | 0.011 | 0.729 |
| D on X (Accuracy) | 0.780 | 0.949 |
+-------------------+-------+-------+


# Naive ML
- no orthogonalisation, no crossfitting

In [61]:
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, y - g_hat)
    return psi_a, psi_b

In [62]:
from doubleml import DoubleMLData
from doubleml import DoubleMLPLR
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
face_colors = sns.color_palette('pastel')
edge_colors = sns.color_palette('dark')
np.random.seed(1111)
ml_l = RandomForestRegressor()
ml_m = RandomForestClassifier()
#ml_m = LogisticRegression()
ml_g = clone(ml_l)

# to speed up the illustration we hard-code the simulation results
theta_nonorth = np.empty(n_rep)
se_nonorth = np.empty(n_rep)
t_nonorth = np.empty(n_rep)
p_nonorth = np.empty(n_rep)
# to run the full simulation uncomment the following line to fit the model for every dataset and not just for the first dataset
for i_rep in range(n_rep):
    print(i_rep)
#for i_rep in range(1):
    (x, y, d) = data[i_rep]
    obj_dml_data = DoubleMLData.from_arrays(x, y.ravel(), d.ravel())
    print(obj_dml_data)
    obj_dml_plr_nonorth = DoubleMLPLR(obj_dml_data,
                                      ml_l, ml_m, ml_g,
                                      n_folds=1,
                                      apply_cross_fitting=False,
                                      score=non_orth_score)
    obj_dml_plr_nonorth.fit()
    theta_nonorth[i_rep] = obj_dml_plr_nonorth.coef[0]
    se_nonorth[i_rep] = obj_dml_plr_nonorth.se[0]

0
================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: y
Treatment variable(s): ['d']
Covariates: ['X1', 'X2', 'X3']
Instrument variable(s): None
No. Observations: 2825

------------------ DataFrame info    ------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2825 entries, 0 to 2824
Columns: 5 entries, X1 to d
dtypes: float64(5)
memory usage: 110.5 KB



# Orthogonal Machine Learning
- resolves regularization bias, but not overfitting

In [63]:
import numpy as np
np.random.seed(2222)
# to speed up the illustration we hard-code the simulation results
theta_orth_nosplit = np.empty(n_rep)
se_orth_nosplit = np.empty(n_rep)
# to run the full simulation uncomment the following line to fit the model for every dataset and not just for the first dataset
for i_rep in range(n_rep):
    print(i_rep)
#for i_rep in range(1):
    (x, y, d) = data[i_rep]
    obj_dml_data = DoubleMLData.from_arrays(x, y, d)
    obj_dml_plr_orth_nosplit = DoubleMLPLR(obj_dml_data,
                                           ml_l, ml_m, ml_g,
                                           n_folds=1,
                                           score='IV-type',
                                           apply_cross_fitting=False)
    obj_dml_plr_orth_nosplit.fit()
    theta_orth_nosplit[i_rep] = obj_dml_plr_orth_nosplit.coef[0]
    se_orth_nosplit[i_rep] = obj_dml_plr_orth_nosplit.se[0]

0


# Orthogonal ML + Cross fitting (DML)

In [64]:
np.random.seed(42)
# to speed up the illustration we hard-code the simulation results
theta_dml = np.empty(n_rep)
se_dml = np.empty(n_rep)

# to run the full simulation uncomment the following line to fit the model for every dataset and not just for the first dataset
for i_rep in range(n_rep):
    print(i_rep)
    (x, y, d) = data[i_rep]
    obj_dml_data = DoubleMLData.from_arrays(x, y, d)
    obj_dml_plr = DoubleMLPLR(obj_dml_data,
                              ml_l, ml_m, ml_g,
                              n_folds=2,
                              score='IV-type')
    obj_dml_plr.fit()
    theta_dml[i_rep] = obj_dml_plr.coef[0]
    se_dml[i_rep] = obj_dml_plr.se[0]

0


# Regular OLS

In [65]:
np.random.seed(42)
import statsmodels.api as sm # for OLS 
# to speed up the illustration we hard-code the simulation results
theta_ols = np.empty(n_rep)
se_ols = np.empty(n_rep)
# to run the full simulation uncomment the following line to fit the model for every dataset and not just for the first dataset
for i_rep in range(n_rep):
    (x, y, d) = data[i_rep]
    OLS = sm.OLS(y,sm.add_constant(np.c_[d,x]))
    results = OLS.fit()
    theta_ols[i_rep] = results.params[1]
    se_ols[i_rep] = results.bse[1]   

# Distribution of Theta

In [66]:
from prettytable import PrettyTable
from PIL import Image, ImageDraw, ImageFont
MC_θ = np.c_[theta_ols, theta_nonorth, theta_orth_nosplit, theta_dml]
MC_se = np.c_[se_ols, se_nonorth, se_orth_nosplit, se_dml]

table = PrettyTable()
table.field_names = ['Estimator', 'θ_hat', 's.e(θ_hat)','t','p','2.5%','97.25%']
a = ['OLS']+ np.c_[results.params[1], results.bse[1], results.tvalues[1], results.pvalues[1], results.conf_int(alpha=0.05, cols=None)[0][1], results.conf_int(alpha=0.05, cols=None)[1][1]].reshape(-1).tolist()
table.add_row(a)
a = ['Naive-ML']+ np.array(obj_dml_plr_nonorth.summary).reshape(-1).tolist()
table.add_row(a)
a = ['Ortho-ML']+ np.array(obj_dml_plr_orth_nosplit.summary).reshape(-1).tolist()
table.add_row(a)
a = ['OrthoML+Crossfitting (DML)']+ np.array(obj_dml_plr.summary).reshape(-1).tolist()
table.add_row(a)
table.float_format = '0.3'
print(table)

+----------------------------+-------+------------+---------+-------+-------+--------+
|         Estimator          | θ_hat | s.e(θ_hat) |    t    |   p   |  2.5% | 97.25% |
+----------------------------+-------+------------+---------+-------+-------+--------+
|            OLS             | 0.450 |   0.019    |  23.560 | 0.000 | 0.413 | 0.488  |
|          Naive-ML          | 0.398 |   0.004    | 105.577 | 0.000 | 0.390 | 0.405  |
|          Ortho-ML          | 0.410 |   0.010    |  39.351 | 0.000 | 0.390 | 0.431  |
| OrthoML+Crossfitting (DML) | 0.455 |   0.025    |  18.407 | 0.000 | 0.407 | 0.503  |
+----------------------------+-------+------------+---------+-------+-------+--------+


# First Stage Checks

In [55]:
from sklearn.linear_model import LinearRegression, LogisticRegression
OLS_Y = LinearRegression()
OLS_D = LogisticRegression()
RF_Y = RandomForestRegressor()
RF_D = RandomForestClassifier()

from prettytable import PrettyTable
from PIL import Image, ImageDraw, ImageFont
MC_θ = np.c_[theta_ols, theta_nonorth, theta_orth_nosplit, theta_dml]
MC_se = np.c_[se_ols, se_nonorth, se_orth_nosplit, se_dml]
table = PrettyTable()
table.field_names = ['Model', 'OLS', 'RF']
a = ['Y on X (R2)'] + [OLS_Y.fit(x,y).score(x,y), RF_Y.fit(x,y).score(x,y)]
table.add_row(a)
a = ['D on X (Accuracy)'] + [OLS_D.fit(x,d).score(x,d), RF_D.fit(x,d).score(x,d)]
table.add_row(a)
table.float_format = '0.3'
print(table)

+-------------------+-------+-------+
|       Model       |  OLS  |   RF  |
+-------------------+-------+-------+
|    Y on X (R2)    | 0.010 | 0.817 |
| D on X (Accuracy) | 0.780 | 0.993 |
+-------------------+-------+-------+


In [56]:
from prettytable import PrettyTable
from PIL import Image, ImageDraw, ImageFont
MC_θ = np.c_[theta_ols, theta_nonorth, theta_orth_nosplit, theta_dml]
MC_se = np.c_[se_ols, se_nonorth, se_orth_nosplit, se_dml]
table = PrettyTable()
table.field_names = ['Model', 'OLS', 'RF']
a = ['Y on X (R2)'] + [OLS_Y.fit(x,y).score(x,y), RF_Y.fit(x,y).score(x,y)]
table.add_row(a)
a = ['D on X (Accuracy)'] + [OLS_D.fit(x,d).score(x,d), RF_D.fit(x,d).score(x,d)]
table.add_row(a)
table.float_format = '0.3'
print(table)

+-------------------+-------+-------+
|       Model       |  OLS  |   RF  |
+-------------------+-------+-------+
|    Y on X (R2)    | 0.010 | 0.819 |
| D on X (Accuracy) | 0.780 | 0.993 |
+-------------------+-------+-------+
